In [14]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

## Khởi tạo model dịch

In [16]:
translate_model_name = "VietAI/envit5-translation"
translate_tokenizer = AutoTokenizer.from_pretrained(translate_model_name)
translate_model = AutoModelForSeq2SeqLM.from_pretrained(translate_model_name)
translate_model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(50048, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50048, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [17]:
inputs = [
    "vi: VietAI là tổ chức phi lợi nhuận với sứ mệnh ươm mầm tài năng về trí tuệ nhân tạo và xây dựng một cộng đồng các chuyên gia trong lĩnh vực trí tuệ nhân tạo đẳng cấp quốc tế tại Việt Nam.",
    "vi: Theo báo cáo mới nhất của Linkedin về danh sách việc làm triển vọng với mức lương hấp dẫn năm 2020, các chức danh công việc liên quan đến AI như Chuyên gia AI (Artificial Intelligence Specialist), Kỹ sư ML (Machine Learning Engineer) đều xếp thứ hạng cao.",
    "en: Our teams aspire to make discoveries that impact everyone, and core to our approach is sharing our research and tools to fuel progress in the field.",
    "en: We're on a journey to advance and democratize artificial intelligence through open source and open science."
]

In [18]:
def translate_texts(sentences):
    outputs = translate_model.generate(translate_tokenizer(sentences, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=512)
    sentences = translate_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(sentences)


In [19]:
translate_texts(inputs)

['en: VietAI is a non-profit organization with the mission of nurturing artificial intelligence talents and building an international - class community of artificial intelligence experts in Vietnam.', 'en: According to the latest LinkedIn report on the 2020 list of attractive and promising jobs, AI - related job titles such as AI Specialist, ML Engineer and ML Engineer all rank high.', 'vi: Nhóm chúng tôi khao khát tạo ra những khám phá có ảnh hưởng đến mọi người, và cốt lõi trong cách tiếp cận của chúng tôi là chia sẻ nghiên cứu và công cụ để thúc đẩy sự tiến bộ trong lĩnh vực này.', 'vi: Chúng ta đang trên hành trình tiến bộ và dân chủ hoá trí tuệ nhân tạo thông qua mã nguồn mở và khoa học mở.']


# Lưu trạng thái model dịch

In [20]:
import torch

In [21]:
torch.save(translate_model.state_dict(), "./translate_model.pth")

## Khởi tạo model rút gọn

In [ ]:
summary_tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-large-vietnews-summarization")  
summary_model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-large-vietnews-summarization")
summary_model.cuda()

Lưu trạng thái model dịch

### Hàm rút gọn

In [ ]:
def summary_text(sentence):
    text =  "vietnews: " + sentence + " </s>"
    encoding = summary_tokenizer(text, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
    sum_outputs = summary_model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        max_length=256,
        early_stopping=True
    )
    for output in sum_outputs:
        line = summary_tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        return line
        

In [ ]:
for i in sentences:
    print(summary_text(i))